In [403]:
import re
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [404]:

# Function to parse the log file
def parse_log_file(log_file_path):
    iterations = []
    mious = []
    maccs = []
    learning_rates = []

    last_iter = None
    last_lr = None

    with open(log_file_path, 'r') as f:
        for line in f:
            # Parse training iteration and learning rate
            train_match = re.search(r"Iter\(train\)\s*\[\s*(\d+)/\d+\]\s*lr:\s*([0-9\.e+-]+)", line)
            if train_match:
                last_iter = int(train_match.group(1))
                last_lr = float(train_match.group(2))
                continue

            # Parse validation metrics (mIoU and mAcc)
            val_match = re.search(r"mIoU:\s*(\d+\.\d+)\s*mAcc:\s*(\d+\.\d+)", line)
            if val_match:
                if last_iter is not None:
                    iterations.append(last_iter)
                    mious.append(float(val_match.group(1)))
                    maccs.append(float(val_match.group(2)))
                    learning_rates.append(last_lr)

    return iterations, mious, maccs, learning_rates

# New function to plot interactively with Plotly
def plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, x_range):
    # Create a figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add Top-1 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=mAcc, name='Mean Accuracy', line=dict(color='blue')),
        secondary_y=False,
    )

    # Add Top-5 Accuracy trace
    fig.add_trace(
        go.Scatter(x=epochs, y=mIoU, name='Mean IoU', line=dict(color='orange')),
        secondary_y=False,
    )

    # Add Learning Rate trace on secondary y-axis
    fig.add_trace(
        go.Scatter(x=epochs, y=lrs, name='Learning Rate', line=dict(color='green')),
        secondary_y=True,
    )

    # Update layout for interactivity and axes
    fig.update_layout(
        title='Validation Performance and Training Learning Rate vs Epoch',
        xaxis_title='Epoch',
        yaxis_title='Performance (%)',
        yaxis2_title='Learning Rate',
        hovermode='x unified',  # Vertical crosshair on x-hover, shows values for all lines
        legend=dict(yanchor='top', y=1.15, xanchor='right', x=1.15),  # Top-right legend
    )

    # Set x-axis ranges
    fig.update_xaxes(range=x_range)
    
    # Set y-axis ranges
    fig.update_yaxes(range=[0, 100], dtick=10, secondary_y=False)
    fig.update_yaxes(secondary_y=True)  # Auto-range for LR

    # Show the figure (interactive in Jupyter)
    fig.show()


In [405]:
adam_range = [0, 310]
sgd_range = [0, 120000]

In [406]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6, Error: Wrong Training Loop, so the learning rate stayed constant throughout
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes/20250814_203128/20250814_203128.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [407]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 8, Error: Wrong Training Loop, so the learning rate stayed constant throughout
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes/20250814_203149/20250814_203149.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [408]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6, 120K - 65.08 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce/20250819_174704/20250819_174704.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [409]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6 - 65.08 Best mIoU. Continued for extra 40k iters
# Finetuning for an extra 40k iterations may not work. This starts the learning rate schedule over, so 
# minima exploration jumps are larger. Going to keep training just for observation though.
# 65.35 Best mIoU 
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce_scratch/20250821_114549/20250821_114549.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,40000])

In [410]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 6, 160K - 66.74 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce/20250821_121920/20250821_121920.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [411]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 8 - 65.85 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ce/20250819_175241/20250819_175241.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [412]:
# SEBNet BaselineHead SGD, Scratch, Batch Size 8 - 65.54 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ce_scratch/20250819_174850/20250819_174850.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [413]:
# SEBNet BaselineHead SGD, Pretrained, Batch Size 8, OHEM Cross Entropy - 67.23 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ohem/20250819_174814/20250819_174814.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [414]:
# SEBNet PIDNet SGD, Pretrained, Batch Size 6
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-bas-head_1xb6_cityscapes/20250821_094855/20250821_094855.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, sgd_range)

In [415]:
# Ablation 01 - SEBNet BaselineHead SGD, Pretrained, Batch Size 6, CE Cross Entropy - 72.15 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce/20250822_155456/20250822_155456.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [416]:
# Ablation 01 - SEBNet BaselineHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 72.26 mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb6_cityscapes_ce_scratch/20250822_155431/20250822_155431.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [417]:
# Ablation 01 - SEBNet BaselineHead SGD, Pretrained, Batch Size 8, CE Cross Entropy - 71.01 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ce/20250822_155554/20250822_155554.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,120000])

In [418]:
# Ablation 01 - SEBNet BaselineHead SGD, Scratch, Batch Size 8, CE Cross Entropy - 70.99 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-head_1xb8_cityscapes_ce_scratch/20250822_155524/20250822_155524.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,120000])

In [419]:
# SEBNet BaselineCASENetHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 72.81 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-casenet-head_1xb6_cityscapes/20250823_235330/20250823_235330.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [420]:
# SEBNet BaselineDFFHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 73.55 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-dff-head_1xb6_cityscapes/20250823_235446/20250823_235446.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [421]:
# SEBNet BaselineBEMHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 73.94 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-bem-head_1xb6_cityscapes/20250823_235417/20250823_235417.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [422]:
# SEBNet BaselineMIMIRHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 73.63 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-mimir-head_1xb6_cityscapes/20250823_235809/20250823_235809.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [423]:
# Ablation 04 - SEBNet BaselineCASENetHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 70.95 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-casenet-head_1xb6_cityscapes/20250826_134858/20250826_134858.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [424]:
# Ablation 05 - SEBNet BaselineDFFHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 72.52 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-dff-head_1xb6_cityscapes/20250826_134936/20250826_134936.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [425]:
# Ablation 06 - SEBNet BaselineBEMHead SGD, Scratch, Batch Size 6, CE Cross Entropy - 73.89 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-bem-head_1xb6_cityscapes/20250826_134919/20250826_134919.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [426]:
# Ablation 07 - SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 74.76 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250826_134843/20250826_134843.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [427]:
# Ablation 13 - SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 1, Loss Weight 5.0 - 74.25 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250829_142153/20250829_142153.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [428]:
# Ablation 14 - SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 4, Loss Weight 5.0 - 74.69 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250829_142216/20250829_142216.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [429]:
# Ablation 15 - SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 8, Loss Weight 5.0 - 74.42 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250829_142252/20250829_142252.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [430]:
# SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 16, Loss Weight 5.0 - 73.91 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250829_142316/20250829_142316.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [431]:
# Ablation 16 - SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 1.0 - 73.65 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250831_172317/20250831_172317.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [432]:
# Ablation 17 - SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 10.0 - 74.14 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250831_172344/20250831_172344.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [433]:
# Ablation 18 - SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 20.0 - 74.20 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250831_172400/20250831_172400.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [434]:
# SEBNet BaselineDMultiLabelHead SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 50.0 - 73.11 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-d-multilabel-head_1xb6_cityscapes/20250831_172417/20250831_172417.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [435]:
# SEBNet ConditionedBaselinePSBD SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-head-conditioned_1xb6_cityscapes/20250903_132832/20250903_132832.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [436]:
# SEBNet BagBaselinePSBD SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-head-fused_1xb6_cityscapes/20250903_132852/20250903_132852.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [437]:
# SEBNet ConditionedBaselinePSBDBAS SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_1xb6_cityscapes/20250903_132914/20250903_132914.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [438]:
# SEBNet BagBaselinePSBDBAS SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-fused_1xb6_cityscapes/20250903_132932/20250903_132932.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [439]:
# Ablation 21 - SEBNet ConditionedBaselinePSBD SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 79.63 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-head-conditioned_1xb6_cityscapes/20250903_224025/20250903_224025.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [440]:
# Ablation 22 - SEBNet BagBaselinePSBD SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 79.24 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-head-fused_1xb6_cityscapes/20250903_224120/20250903_224120.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [441]:
# Ablation 23 - SEBNet ConditionedBaselinePSBDBAS SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 80.05 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_1xb6_cityscapes/20250903_224209/20250903_224209.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [442]:
# Ablation 24 - SEBNet BagBaselinePSBDBAS SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 80.03 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-fused_1xb6_cityscapes/20250903_224257/20250903_224257.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [443]:
# SEBNet ConditionedBaselinePSBDBAS SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 79.99 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_1xb6_cityscapes/20250906_102636/20250906_102636.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [444]:
# Ablation 23 - SEBNet ConditionedBaselinePSBDBAS SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 80.7 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_1xb6_cityscapes/20250906_102650/20250906_102650.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [445]:
# SEBNet BaselinePDSBDBAS SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 80.62 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_cityscapes/20250906_102527/20250906_102527.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [446]:
# Ablation 27 - SEBNet BaselinePDSBDBAS SGD, Scratch, Batch Size 6, CE Cross Entropy, Edge Size 2, Loss Weight 5.0 - 81.06 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_cityscapes/20250906_102604/20250906_102604.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240064])

In [447]:
# SEBNet ConditionedBaselinePSBDBAS SGD, Mapillary, 240K Iters - 28.95 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_1xb6_mapillaryv2/20250909_152750/20250909_152750.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240000])

In [448]:
# SEBNet ConditionedBaselinePSBDBAS SGD, Mapillary, 360K Iters - 23.64 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_1xb6_mapillaryv2/20250909_152839/20250909_152839.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,360000])

In [449]:
# SEBNet BaselinePDSBDBAS SGD, Mapillary, 240K Iters - 30.40 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_mapillaryv2/20250909_152722/20250909_152722.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,240000])

In [450]:
# SEBNet BaselinePDSBDBAS SGD, Mapillary, 360K Iters - 25.05 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_1xb6_mapillaryv2/20250909_152641/20250909_152641.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,360000])

In [451]:
# SEBNet ConditionedBaselinePSBDBAS SGD, Mapillary, LR 1e-2, 1.4M Iters, 1 GPU - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_lr-1e-2_1xb6_mapillaryv2/20250915_204136/20250915_204136.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,1452000])

In [452]:
# SEBNet ConditionedBaselinePSBDBAS SGD, Mapillary, LR 5e-3, 1.4M Iters, 1 GPU - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_lr-5e-3_1xb6_mapillaryv2/20250915_204155/20250915_204155.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,1452000])

In [453]:
# SEBNet BaselinePDSBDBAS SGD, Mapillary, LR 1e-2, 1.4M Iters, 1 GPU - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_lr-1e-2_1xb6_mapillaryv2/20250915_204029/20250915_204029.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,1452000])

In [454]:
# SEBNet BaselinePDSBDBAS SGD, Mapillary, LR 5e-3, 1.4M Iters, 1 GPU - XX.XX Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_lr-5e-3_1xb6_mapillaryv2/20250915_204116/20250915_204116.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,1452000])

In [455]:
# Ablation 28 - SEBNet BaselinePDSBDBAS SGD, Mapillary, LR 1e-2, 726K Iters, 2 GPUs - 37.12 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-d-sbd-bas-head_2xb6_mapillaryv2/20250916_135803/20250916_135803.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,726000])

In [456]:
# Ablation 28 - SEBNet ConditionedBaselinePSBDBAS SGD, Mapillary, LR 1e-2, 726K Iters, 2 GPUs - 36.65 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_baseline-p-sbd-bas-head-conditioned_2xb6_mapillaryv2/20250916_155205/20250916_155205.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,726000])

In [457]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 20.0, SBD: 1.0 - 73.68 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251007_165817/20251007_165817.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [458]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 20.0, SBD: 10.0 - 73.84 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251007_165909/20251007_165909.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [459]:
# Ablation 41 - 160K Iters, Batch Size 6, HED: 20.0, SBD: 20.0 - 74.01 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation41_1xb6-160k_cityscapes/20251007_165925/20251007_165925.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [460]:
# Ablation 24 - 160K Iters, Batch Size 6 - 79.41 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation24_1xb6-160k_cityscapes/20251009_185601/20251009_185601.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [461]:
# Ablation 30 - 160K Iters, Batch Size 6 - 79.75 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation30_1xb6-160k_cityscapes/20251009_185626/20251009_185626.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [462]:
# Ablation 36 - 160K Iters, Batch Size 6 - 80.08 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation36_1xb6-160k_cityscapes/20251009_185644/20251009_185644.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [463]:
# Ablation 39 - 160K Iters, Batch Size 6 - 80.01 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation39_1xb6-160k_cityscapes/20251009_185657/20251009_185657.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [464]:
# Ablation 48 - 160K Iters, Batch Size 6 - 74.01 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation48_1xb6-160k_cityscapes/20251010_091716/20251010_091716.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [465]:
# Ablation 49 - 160K Iters, Batch Size 6 - 79.61 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation49_1xb6-160k_cityscapes/20251010_091735/20251010_091735.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,160000])

In [466]:
# Ablation 49 - 241K Iters, Batch Size 6 - 79.61 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation49_1xb6-160k_cityscapes/20251013_095259/20251013_095259.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,241000])

In [467]:
# Ablation 40 - 241K Iters, Batch Size 6 - 79.61 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation40_1xb6-160k_cityscapes/20251013_134001/20251013_134001.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,241000])

In [468]:
# Ablation 33 - 241K Iters, Batch Size 6 - 79.61 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/sebnet_ablation33_1xb6-160k_cityscapes/20251013_133840/20251013_133840.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,241000])

In [469]:
# PIDNet-L - 240K - 79.89 Best mIoU
log_file_path = '/home/robert.breslin/alessandro/testing/paper_2/mmsegmentation/work_dirs/pidnet-l_1xb6-241k_1024x1024-cityscapes/20251014_190556/20251014_190556.log'
epochs, mIoU, mAcc, lrs = parse_log_file(log_file_path)
plot_accuracies_and_lr_interactive(epochs, mAcc, mIoU, lrs, [0,241000])